In [182]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix
from datetime import datetime
import joblib


In [183]:
# Load the dataset
df = pd.read_csv('masterdata.csv')

# Convert 'game_date' column to datetime objects
df['game_date'] = pd.to_datetime(df['game_date'])

# Get today's date
today = datetime.today().strftime('%Y-%m-%d')

# Filter out rows with today's date
df = df[df['game_date'] != today]

# Drop the columns containing 'Name', 'ID', or '_P_'
columns_to_drop = [col for col in df.columns if 'Name' in col or 'ID' in col or '_P_' in col]
df = df.drop(columns=columns_to_drop)

# Define features and target variable
X = df.drop(columns=['over_under_target', 'runs_total', 'game_date', 'runs_home', 'runs_away', 'game_id', 'home_name', 'away_name'])
y = df['over_under_target']

# You can now proceed to train your model using X and y
# Example: Train a RandomForestClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

# Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42)

# Initialize and train the model
model = RandomForestClassifier(random_state=42)
model.fit(X_train, y_train)

# Make predictions on the test set
y_pred = model.predict(X_test)

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
conf_matrix = confusion_matrix(y_test, y_pred)
class_report = classification_report(y_test, y_pred)

print(f'Accuracy: {accuracy}')
print('Confusion Matrix:\n', conf_matrix)
print('Classification Report:\n', class_report)


C:\Users\kesse\AppData\Local\Temp\ipykernel_20308\224154518.py:2: DtypeWarning: Columns (492,570,571,598,599,626,627,663,775,853,854,881,882) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('masterdata.csv')


Accuracy: 0.5954198473282443
Confusion Matrix:
 [[224 142]
 [176 244]]
Classification Report:
               precision    recall  f1-score   support

         0.0       0.56      0.61      0.58       366
         1.0       0.63      0.58      0.61       420

    accuracy                           0.60       786
   macro avg       0.60      0.60      0.60       786
weighted avg       0.60      0.60      0.60       786



In [184]:
print(len(y_test), len(y_train))

786 7068


## Save Model

In [185]:
# Save the model
joblib.dump(model, 'model.pkl')

['model.pkl']

## Predict Games

In [186]:
# Load the dataset
df = pd.read_csv('masterdata.csv')

# Convert 'game_date' column to datetime objects
df['game_date'] = pd.to_datetime(df['game_date'])


# Get today's date
today = datetime.today().strftime('%Y-%m-%d')

# Separate the data for today's games
todays_games = df[df['game_date'] == today]

# Check if there are any games today
if todays_games.empty:
    print("No games found for today.")
else:
    # Define the columns to drop
    columns_to_drop = [col for col in df.columns if 'Name' in col or 'ID' in col or '_P_' in col]
    columns_to_drop.extend(['over_under_target', 'runs_total', 'game_date', 'runs_home', 'runs_away', 'game_id', 'home_name', 'away_name'])

    # Drop the unnecessary columns
    X_todays_games = todays_games.drop(columns=columns_to_drop)

    # Load the trained model (assuming it's saved as 'model.pkl')
    model = joblib.load('model.pkl')



C:\Users\kesse\AppData\Local\Temp\ipykernel_20308\4035638571.py:2: DtypeWarning: Columns (492,570,571,598,599,626,627,663,775,853,854,881,882) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('masterdata.csv')


In [187]:
# Make predictions
predictions = model.predict(X_todays_games)

In [188]:
print(predictions)

[1. 0. 1. 1. 1. 1. 0. 0. 1. 0. 1. 0. 1. 1. 0.]


In [189]:
# Interpret and display the predictions
todays_games['prediction'] = predictions
for i, row in todays_games.iterrows():
    result = 'Over' if row['prediction'] == 1 else 'Under'
    print(f"Game {i + 1}: {result} the runline")

Game 7855: Over the runline
Game 7856: Under the runline
Game 7857: Over the runline
Game 7858: Over the runline
Game 7859: Over the runline
Game 7860: Over the runline
Game 7861: Under the runline
Game 7862: Under the runline
Game 7863: Over the runline
Game 7864: Under the runline
Game 7865: Over the runline
Game 7866: Under the runline
Game 7867: Over the runline
Game 7868: Over the runline
Game 7869: Under the runline


C:\Users\kesse\AppData\Local\Temp\ipykernel_20308\3339359323.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  todays_games['prediction'] = predictions
